Importing libraries

In [ ]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
from skimage.util import montage
from skimage.segmentation import mark_boundaries
import tensorflow as tf

In [ ]:
train_image_dir = r"D:\Ship detection dataset\train_v2"
train_encode_file = r"D:\Ship detection dataset\train_ship_segmentations_v2.csv"
test_image_dir = r"D:\Ship detection dataset\test_v2"

Train directory

In [ ]:
train_images = os.listdir(train_image_dir)
print(f"Total {len(train_images)} images in the train directory. \nHere are the first 5 images: - {train_images[:5]}")

Test directory

In [ ]:
test_images = os.listdir(test_image_dir)
print(f"Total {len(test_images)} images in the test directory. \nHere are the first 5 images: - {test_images[:5]}")

Visualize some test images

In [ ]:
plt.figure(figsize=(15, 15))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(cv2.imread(test_image_dir + '/' + test_images[i]))
    plt.title(f"{test_images[i]}", weight='bold')
    plt.axis('off')
plt.tight_layout()

Read the CSV file

In [ ]:
df = pd.read_csv(train_encode_file)

In [ ]:
df.info()  # Some information about train_encode_file

In [ ]:
df.describe()

In [ ]:
df.head(10)

In [ ]:
df[df["ImageId"] == "00021ddc3.jpg"]  # In df, for the same image, we have more than one ship, and each ship is in a separate row.

Build a dictionary where the key is the image id, and the value is the number of ships in the image

In [ ]:
ship_num = {}

In [ ]:
for index, row in df.iterrows():
    image_id = row['ImageId']
    encoding = row['EncodedPixels']
    if pd.isna(encoding) or encoding == "":
        ship_num[image_id] = 0
    else:
        if image_id in ship_num:
            ship_num[image_id] += 1
        else:
            ship_num[image_id] = 1

In [ ]:
print(ship_num)

In [ ]:
img = cv2.imread(train_image_dir + "/30d3f7721.jpg")
plt.imshow(img)
print(f"Number of ships in this image is: {ship_num['30d3f7721.jpg']}")  # Verify the number of ships for the image with id 30d3f7721.jpg

In [ ]:
print(len(ship_num))
# Coincides with the unique number of elements
# Number of images

Transform the dictionary into a DataFrame for easier information manipulation

In [ ]:
MonData = pd.DataFrame(list(ship_num.items()), columns=['ImageId', 'ships'])

In [ ]:
MonData.head(10)

We verified that the image with id 00003e153.jpg does not have any ships

In [ ]:
img = cv2.imread(train_image_dir + "/00003e153.jpg")
plt.imshow(img)
print("This image with id \"00003e153.jpg\" does not represent any ship according to the previous table.")

In [ ]:
plt.figure(figsize=(12, 8))  # Adjust the figure size
sns.countplot(data=MonData, x='ships', palette='Set2')
# Add labels and a title
plt.title('Distribution of the number of images', fontsize=16)
plt.xlabel('Number of ships', fontsize=14)
plt.ylabel('Number of occurrences', fontsize=14)
plt.show()

In [ ]:
MonData['ships'].value_counts()  # Number of images with the same number of ships
# We can observe an imbalance in the distribution of images based on the number of ships.

List of image paths

In [ ]:
image_paths = [
    os.path.join(train_image_dir, filename) for filename in os.listdir(train_image_dir)
]

In [ ]:
image_paths[10:20]

In [ ]:
print(df.isnull().sum())  # Check for missing values

Detailed analysis of images

In [ ]:
for i in range(5):
    image = cv2.imread(image_paths[i])
    height, width, channels = image.shape  # Get the dimensions of the image
    image_size = height * width  # Calculate the size of the image
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"Image {i + 1}")
    plt.axis('off')
    plt.show()
    print(f"Image dimensions: {height} x {width} pixels")
    print(f"Number of channels: {channels}")
    print(f"Image size: {image_size} pixels")

In [ ]:
print('Number of images in CSV:', len(df))
print('Number of images in the image file:', len(image_paths))

In [ ]:
number_images_without_ship = 0
number_images_with_ship = 0

Iterate through the ship_num dictionary

In [ ]:
for number_ships in ship_num.values():
    if number_ships == 0:
        number_images_without_ship += 1
    else:
        number_images_with_ship += 1

Create two lists for the x and y axes of the graph

In [ ]:
categories = ['Without Ship', 'With Ship']
number_images = [number_images_without_ship, number_images_with_ship]
print("Number of images without ship", number_images_without_ship)
print("Number of images with ship", number_images_with_ship)
# Create a bar chart
plt.bar(categories, number_images)

Label the axes of the graph

In [ ]:
plt.xlabel('Category')
plt.ylabel('Number of images')

Display the graph

In [ ]:
plt.show()

Create a new column 'Size' that contains the number of pixels per ship

In [ ]:
df['Size'] = df['EncodedPixels'].apply(lambda x: 0 if pd.isna(x) else sum(map(int, str(x).split()[1::2])))

Group the data by size and count the number of ships of each size

In [ ]:
ship_sizes = df[df['Size'] > 0]['Size'].value_counts().reset_index()
ship_sizes.columns = ['Ship Size (in pixels)', 'Number of Ships']

Sort the data by size in ascending order

In [ ]:
ship_sizes = ship_sizes.sort_values(by='Ship Size (in pixels)')

Plot the graph

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(ship_sizes['Ship Size (in pixels)'], bins=50, color='skyblue', edgecolor='black')
#plt.bar(ship_sizes['Ship Size (in pixels)'], ship_sizes['Number of Ships'])
plt.title('Number of Ships by Size')
plt.xlabel('Ship Size (in pixels)')
plt.ylabel('Number of Ships')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
df['file_size_kb'] = df['ImageId'].map(
    lambda c_img_id: os.stat(os.path.join(train_image_dir, c_img_id)).st_size / 1024)  # calculate file sizes

In [ ]:
df.head()

In [ ]:
def plot_color_histogram(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert the image from BGR to RGB (Matplotlib uses the RGB format)
    plt.imshow(image_rgb)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()
    r, g, b = cv2.split(image_rgb)  # Split the image into its color channels (Red, Green, Blue)
    plt.figure(figsize=(12, 4))
    # Histogram for the red channel
    plt.subplot(131)
    plt.hist(r.flatten(), bins=256, color='red', alpha=0.7, rwidth=0.8)
    plt.title('Red Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    # Histogram for the green channel
    plt.subplot(132)
    plt.hist(g.flatten(), bins=256, color='green', alpha=0.7, rwidth=0.8)
    plt.title('Green Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')

    # Histogram for the blue channel
    plt.subplot(133)
    plt.hist(b.flatten(), bins=256, color='blue', alpha=0.7, rwidth=0.8)
    plt.title('Blue Histogram')
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

In [ ]:
image_paths[0:]
plot_color_histogram(image_paths[0])
plot_color_histogram(image_paths[10])
plot_color_histogram(image_paths[900])

In [ ]:
def orientation_distribution(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load the image in grayscale
    # Apply the Sobel filter to get the gradients
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
    gradient_orientation = np.arctan2(sobely, sobelx)  # Calculate the orientation of the gradients
    gradient_orientation_degrees = np.degrees(gradient_orientation)  # Convert the orientation to degrees
    flattened_orientation = gradient_orientation_degrees.flatten()  # Flatten the array for the histogram
    # Show the original image
    plt.subplot(121)
    plt.imshow(image, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    # Show the histogram of orientations
    plt.subplot(122)
    plt.hist(flattened_orientation, bins=36, range=[-180, 180], color='black', alpha=0.7)
    plt.title('Orientation Histogram')
    plt.xlabel('Orientation in degrees')
    plt.ylabel('Frequency')
    plt.tight_layout()  # Automatically adjust spaces to avoid overlap
    plt.show()

In [ ]:
orientation_distribution(image_paths[10])


<br>
**The maximum frequency of the diagram is 17500, corresponding to an orientation of 100 degrees. This suggests that the dominant orientation of objects in the image is 100 degrees, which corresponds to the orientation of the ship.**<br>


In [ ]:
for i in range(1, 5):
    orientation_distribution(image_paths[i])

In [ ]:
def detect_keypoints(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()  # Initialize the ORB extractor
    keypoints, descriptors = orb.detectAndCompute(gray_image, None)  # Detect keypoints and descriptors with ORB
    image_with_keypoints = cv2.drawKeypoints(image, keypoints, None)  # Draw keypoints on the image
    # Display the image with keypoints
    plt.imshow(cv2.cvtColor(image_with_keypoints, cv2.COLOR_BGR2RGB))
    plt.title('Keypoint Detection with ORB (Oriented FAST and Rotated BRIEF)')
    plt.axis('off')
    plt.show()

Call the detect_keypoints function for a specific image path

In [ ]:
detect_keypoints(image_paths[10])


<br>
We notice that most of the keypoints obtained by the ORB filter are on the ship.<br>



<br>
Evaluate sharpness and contrast of an image.<br>


In [ ]:
def evaluate_sharpness_contrast(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray_image, cv2.CV_64F)  # Calculate the Laplacian filter to evaluate sharpness
    sharpness = np.var(laplacian)  # Calculate the variance of the Laplacian image for sharpness
    contrast = np.max(gray_image) - np.min(gray_image)  # Calculate the contrast of the image
    # Normalize values between 0 and 1
    sharpness_normalized = sharpness / (sharpness + contrast)
    contrast_normalized = contrast / (sharpness + contrast)
    # Produce a normalized overall score
    score_normalized = (sharpness_normalized + contrast_normalized) / 2
    return score_normalized


<br>
There are 192,225 examples; analyzing all images would be too time-consuming. Focus on a sample of 4000 images to maintain efficiency.<br>


In [ ]:
contrast = set()  # Initialize a set
for i in range(4000):
    pt = image_paths[i]
    score = evaluate_sharpness_contrast(pt)
    contrast.add(score)

Display the contrast scores

In [ ]:
contrast


<br>
We notice that all images have the same contrast and sharpness score.<br>


In [ ]:
shape_unique = set()
for i in range(4000):
    pt = image_paths[i]
    image = cv2.imread(image_paths[i])
    shape_unique.add(image.shape)

Display the unique shapes of images

In [ ]:
shape_unique


<br>
Exploratory Data Analysis (EDA) provided an initial understanding of image features. Now, we will focus on specific data preprocessing techniques, including segmentation by a binary mask, to extract finer information on regions of interest. This is crucial as segmentation allows for more precise ship detection, especially in a uniform distribution of water, considering that ship positions are given in the CSV file. We will also explore model selection and hyperparameter optimization, which will be a U-Net for the moment, and don't forget about data augmentation.<br>


Data Preprocessing

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
def masks_image(in_mask_list):
    # Display an example of a training image and its mask
    all_masks = np.zeros((768, 768), dtype = np.uint8)
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks |= rle_decode(mask)
    return all_masks

Additional data preprocessing steps

In [ ]:
from skimage.io import imread
for num in [100, 88, 5]:
    rle_0 = df.query(f'ImageId=="{train_images[num-1]}"')['EncodedPixels']
    img_0 = masks_image(rle_0)
    original = imread(train_image_dir+"/"+train_images[num-1])
    plt.figure(figsize=(15, 8))
    plt.subplot(1, 2, 1)
    plt.title(f"Original - Train Image {original.shape}")
    plt.imshow(original)
    plt.subplot(1, 2, 2)
    plt.title(f"Mask generated from the RLE data for each ship {img_0.shape}")
    plt.imshow(img_0, cmap = "Blues_r")
    plt.tight_layout()
    plt.show()

In [ ]:
MonData['file_size_kb']=MonData['ImageId'].map(lambda c_img_id:os.stat(os.path.join(train_image_dir,c_img_id)).st_size/1024) # file size calculation
MonData['file_size_kb'].hist()

Filter the data to include only entries with a file size greater than 50 KB

In [ ]:
MonData = MonData[MonData.file_size_kb > 50]  
# Plot a histogram of the 'file_size_kb' column to visualize the distribution of file sizes
MonData['file_size_kb'].hist()
# Select randomly 7 samples from the dataset
MonData.sample(7)  
# Display the shape of the dataset
MonData.shape

Drop the 'Size' column from the DataFrame 'df'

In [ ]:
df.drop(['Size'], axis=1, inplace=True)

In [ ]:
MonData.head()

In [ ]:
from sklearn.model_selection import train_test_split

Split the 'MonData' dataset into training (train) and validation (valid) sets

In [ ]:
train, valid = train_test_split(MonData, test_size=0.2, stratify=MonData['ships'])
print("The shape of the training dataframe is:", train.shape)

Merge the 'df' DataFrame with the 'train' DataFrame

In [ ]:
train_df = pd.merge(df, train)
train_df.sort_values(by='ImageId')  # Sort the resulting DataFrame 'train_df' by the 'ImageId' column
# Display the 'train_df' DataFrame
print(train_df)


The number of rows in 'train' was 153031 after splitting into train and validation, <br>
but after merging, the size of the train dataframe is 184071 rows. <br>
This represents lines from the same image but with different ships, <br>
 the lines in 'df' represent ships and an image can contain multiple ships.


Merge with the 'df' DataFrame to create the 'valid_df' DataFrame for validation

In [ ]:
valid_df = pd.merge(df, valid)
print("The shape of the validation dataframe is:", valid_df.shape)

In [ ]:
print("We find:")
print(train_df.shape[0], 'training masks,')
print(valid_df.shape[0], 'validation masks.')

Plot the ship count distribution in the training data

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=train_df, x='ships', palette='Set2')
plt.show()

The goal of this code is to try to merge the number of ships per image into two classes <br>
to balance the different images

In [ ]:
train_df['grouped_ship_count'] = train_df.ships.map(lambda x: (x+1)//2).clip(0, 7)
train_df.head()

0 is the image with no ships<br>
1 represents images with one ship and images with two ships

In [ ]:
train_df.grouped_ship_count.value_counts()  # Display the count of unique values in the 'grouped_ship_count' column of 'train_df'

Function to sample entries based on the number of ships

In [ ]:
def sample_ships(in_df, base_rep_val=2100):
    if in_df['ships'].values[0] == 0:  # Check if the number of ships is equal to 0
        return in_df.sample(base_rep_val // 3)   # Sample one-third of the value 2100
    else:
        return in_df.sample(base_rep_val)

Create a more or less balanced DataFrame

In [ ]:
balanced_train_df = train_df.groupby('grouped_ship_count').apply(sample_ships)
balanced_train_df.grouped_ship_count.value_counts()
print("The shape of the balanced dataframe is:", balanced_train_df.shape)

For each subgroup from 0 to 7, display the number of training images grouped by the number of ships

In [ ]:
for i in range(8):
    df_val_counts = balanced_train_df[balanced_train_df.grouped_ship_count==i].ships.value_counts()
    print(f"Data frame for grouped ship count = {i}:-\n{df_val_counts}\nSum of Values:- {df_val_counts.values.sum()}\n\n")

Visualize the distribution of training data before and after balancing

In [ ]:
plt.figure(figsize=(15, 5))
plt.suptitle("Train Data", fontsize=18, color='r', weight='bold')

In [ ]:
plt.subplot(1, 2, 1)
sns.countplot(data=train_df, x='ships', palette='Set2')
plt.title("Ship Counts - Before Balancing", fontsize=15)
plt.ylabel("Count", fontsize=13)
plt.xlabel("# Ships in an image", fontsize=13)

In [ ]:
plt.subplot(1, 2, 2)
sns.countplot(data=balanced_train_df, x='ships', palette='Set2')
plt.title("Ship Counts - After Balancing", fontsize=15)
plt.xlabel("# Ships in an image", fontsize=13)
plt.ylabel("Count", fontsize=13)

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
BATCH_SIZE = 48  # Batch size
IMG_SCALING = (3, 3)  # Image resizing parameter

In [ ]:
def make_image_gen(in_df, batch_size=BATCH_SIZE):
    all_batches = list(in_df.groupby('ImageId'))
    out_rgb = []
    out_mask = []
    while True:
        np.random.shuffle(all_batches)  # Shuffle the order of batches
        for c_img_id, c_masks in all_batches:
            # Path to the RGB image
            rgb_path = os.path.join(train_image_dir, c_img_id)
            c_img = imread(rgb_path)  # Load the image
            c_mask = np.expand_dims(masks_image(c_masks['EncodedPixels'].values), -1)  # Create the mask from pixel encodings
            # Resize the image and mask if specified
            if IMG_SCALING is not None:
                c_img = c_img[::IMG_SCALING[0], ::IMG_SCALING[1]]
                c_mask = c_mask[::IMG_SCALING[0], ::IMG_SCALING[1]]

            # Add the image and mask to the lists
            out_rgb += [c_img]
            out_mask += [c_mask]
            # If the batch size is reached, yield the batch
            if len(out_rgb) >= batch_size:
                yield np.stack(out_rgb, 0) / 255.0, np.stack(out_mask, 0)
                out_rgb, out_mask = [], []  # Reset the lists for the next batch

Call the function to generate training images

In [ ]:
train_gen = make_image_gen(balanced_train_df)
# Image and Mask
train_x, train_y = next(train_gen)
print(f"train_x ~\nShape: {train_x.shape}\nMin value: {train_x.min()}\nMax value: {train_x.max()}")
print(f"\ntrain_y ~\nShape: {train_y.shape}\nMin value: {train_y.min()}\nMax value: {train_y.max()}")

Visualize a batch of training data

In [ ]:
from skimage.segmentation import mark_boundaries
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
batch_rgb = montage_rgb(train_x) 
batch_seg = montage(train_y[:, :, :, 0])  
batch_overlap = mark_boundaries(batch_rgb, batch_seg.astype(int)) 
titles = ["Images", "Segmentations", "Ship Contours in Images"]
colors = ['g', 'm', 'b']
display = [batch_rgb, batch_seg, batch_overlap]
plt.figure(figsize=(25, 10))
for i in range(3):
    plt.subplot(1, 3, i+1) 
    plt.imshow(display[i])
    plt.title(titles[i], fontsize=18, color=colors[i])
    plt.axis('off')
plt.suptitle("Batch Visualization", fontsize=20, color='r', weight='bold')
plt.tight_layout()

In [ ]:
VALID_IMG_COUNT = 400 # Set the number of validation images
valid_x, valid_y = next(make_image_gen(valid_df, VALID_IMG_COUNT))
print(f"valid_x ~\nShape: {valid_x.shape}\nvalue min: {valid_x.min()}\nvalue max: {valid_x.max()}")
print(f"\nvalid_y ~\nShape: {valid_y.shape}\n value min: {valid_y.min()}\nvalue max: {valid_y.max()}")

Using ImageDataGenerator for data augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

Parameters for image augmentation

In [ ]:
dg_args = dict(rotation_range=15,          # Range of degrees for random rotations
               horizontal_flip=True,       # Perform random horizontal flips
               vertical_flip=True,         # Perform random vertical flips
               data_format='channels_last')  # channels_last refers to (batch, height, width, channels)

In [ ]:
image_gen = ImageDataGenerator(**dg_args)  # Create an image generator with the specified parameters
label_gen = ImageDataGenerator(**dg_args)  # Create a label generator with the specified parameters

Define a function to create an augmented generator

In [ ]:
def create_aug_gen(in_gen, seed=None):
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    for in_x, in_y in in_gen:
        seed = np.random.choice(range(9999))  # Set a seed to have the same augmentation for both image and mask
        g_x = image_gen.flow(255 * in_x,
                             batch_size=in_x.shape[0],
                             seed=seed,
                             shuffle=False)
        g_y = label_gen.flow(in_y,
                             batch_size=in_x.shape[0],
                             seed=seed,
                             shuffle=False)
        yield next(g_x) / 255.0, next(g_y)  # Normalize the pixel values of the image and label

Augment the training data

In [ ]:
cur_gen = create_aug_gen(train_gen, seed=42)
t_x, t_y = next(cur_gen)

Display information about the augmented data

In [ ]:
print('x', t_x.shape, t_x.dtype, t_x.min(), t_x.max())
print('y', t_y.shape, t_y.dtype, t_y.min(), t_y.max())

In [ ]:
import random

Total number of examples

In [ ]:
total_examples = t_x.shape[0]

Set a seed to make the random selection reproducible

In [ ]:
random_seed = 3
random.seed(random_seed)

Randomly select indices for examples to display

In [ ]:
random_indices = random.sample(range(total_examples), 4)

Display randomly selected examples of images and their masks

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(10, 6))
for i, random_index in enumerate(random_indices):
    # Display the image
    axes[0, i].imshow(t_x[random_index])
    axes[0, i].set_title(f"Image {random_index + 1}")

    # Display the mask
    axes[1, i].imshow(t_y[random_index])
    axes[1, i].set_title(f"Mask {random_index + 1}")

In [ ]:
plt.show()

Define a lambda function to create an RGB montage from an array of images

In [ ]:
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)

Final display before passing the data to the model

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(25, 10))

Display the RGB montage of training images

In [ ]:
ax1.imshow(montage_rgb(t_x), cmap='gray')
ax1.set_title('Images', fontsize=18, color='g')
ax1.axis('off')

Display the montage of masks

In [ ]:
ax2.imshow(montage(t_y[:, :, :, 0]), cmap='Blues_r')
ax2.set_title('Masks', fontsize=18, color='r')
ax2.axis('off')

Display contours around ships in training images

In [ ]:
ax3.imshow(mark_boundaries(montage_rgb(t_x), montage(t_y[:, :, :, 0].astype(int))))
ax3.set_title('Bounding Box', fontsize=18, color='b')
ax3.axis('off')

In [ ]:
plt.tight_layout()

Model Construction

In [ ]:
from keras import models, layers

In [ ]:
def unet(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = layers.Dropout(0.1)(c1)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = layers.Dropout(0.1)(c2)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = layers.Dropout(0.1)(c3)
    c3 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = layers.Dropout(0.1)(c4)
    c4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = layers.Dropout(0.1)(c5)
    c5 = layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    u6 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = layers.Dropout(0.2)(c6)
    c6 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    u7 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = layers.Dropout(0.2)(c7)
    c7 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    u8 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = layers.Dropout(0.2)(c8)
    c8 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    u9 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = layers.Dropout(0.2)(c9)
    c9 = layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    out = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    model = models.Model(inputs=[inputs], outputs=[out])
    model.summary()
    return model

In [ ]:
new_model = unet()

Model layers

In [ ]:
new_model.layers

In [ ]:
import keras.backend as k
import tensorflow as tf

In [ ]:
def dice_coeff(y_pred, y_true):
    y_true = k.cast(y_true, dtype=tf.float32)
    y_pred = k.cast(y_pred, dtype=tf.float32)
    intersection = k.sum(y_pred * y_true, axis=[1, 2, 3])
    union = k.sum(y_pred, axis=[1, 2, 3]) + k.sum(y_true, axis=[1, 2, 3])
    dice_coefficient = k.mean((2. * intersection + 1) / (union + 1))
    return dice_coefficient

In [ ]:
def focal_loss(y_pred, y_true, gamma=2, alpha=0.8):
    y_true = k.cast(y_true, dtype=tf.float32)
    y_pred = k.cast(y_pred, dtype=tf.float32)
    y_pred = k.flatten(y_pred)
    y_true = k.flatten(y_true)
    BCE = k.binary_crossentropy(y_pred, y_true)
    EXP_BCE = k.exp(-BCE)
    focal_loss = k.mean(alpha * k.pow((1 - EXP_BCE), gamma) * BCE)
    return focal_loss

Testing the function that calculates dice_coeff

In [ ]:
dice_coeff_result = dice_coeff(train_x, train_y)
print(dice_coeff_result.numpy())

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
new_model.compile(optimizer=Adam(1e-3, beta_1=1e-6), loss=focal_loss, metrics=[dice_coeff])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
reduce = ReduceLROnPlateau(monitor='val_dice_coeff', factor=0.23,
                           patience=3, verbose=1, mode='max',
                           min_delta=0.0001, cooldown=2, min_lr=1e-6)

In [ ]:
early = EarlyStopping(monitor="val_dice_coeff", mode="max", patience=20)

In [ ]:
callbacks_list = [reduce, early]

In [ ]:
aug_gen = create_aug_gen(train_gen)

In [ ]:
new_model.fit(aug_gen, epochs=200, validation_data=(valid_x, valid_y), steps_per_epoch=30,
                  callbacks=callbacks_list)

In [ ]:
new_model.save('new_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
final_model = load_model('new_model.h5', custom_objects={'focal_loss': focal_loss, 'dice_coeff': dice_coeff})

In [ ]:
def prediction(path, img_id, model):
    img = imread(os.path.join(path, img_id))
    img = img[::3, ::3]
    img = img / 255.0  # Resize the image to the size expected by the model
    img = np.expand_dims(img, axis=0)

    # Make sure to provide data to the predict method
    pred = model.predict(img)
    img = np.squeeze(img, axis=0)
    pred_squeezed = np.squeeze(pred, axis=0)
    return img, pred_squeezed

Model evaluation

In [ ]:
print(cv2.imread(r"D:\Ship detection dataset\test_v2\00002bd58.jpg").shape)
for sample in range(20):
    img, pred = prediction(r"D:\Ship detection dataset\test_v2", test_images[sample], final_model)
    fig = plt.figure(figsize=(8, 8))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img)
    plt.axis('off')
    fig.add_subplot(1, 2, 2)
    plt.imshow(pred)
    plt.axis('off')